In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [2]:
classifiers = ['NB', 'DT', 'RF']

## Import test Results

In [10]:
def get_results(path):
    df = pd.read_csv(path)
    
    class_name_pair = np.unique([ (t,f'{p}') for t,p in df[['class', 'label']].values ], axis=0)
    classes = { k:v for v,k in class_name_pair }
    
    metrics = classification_report(df.label, df.prediction, output_dict=True)
    rows = []
    for k in sorted(classes.keys(), key=lambda x: float(x)):
        r = (classes[k], metrics[k]['precision'], metrics[k]['recall'], metrics[k]['f1-score'], metrics[k]['support'])
        rows.append(r)
        
    df_metrics = pd.DataFrame(rows, columns=['class',  'precision', 'recall', 'f1-score', 'support'])
    
    return df_metrics, metrics

In [13]:
metrics = []
for cls_ in classifiers:
    path = glob.glob(f'./results/{cls_}/*csv')[0]
    mc, m = get_results(path)
    metrics.append(m)
#     mc.to_clipboard()
#     print(cls_)
#     input()

/home/k4t0mono/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4t0mono/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4t0mono/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4t0mono/.local/lib/python3.

In [20]:
rows = []
for i, m in enumerate(metrics):
    r = classifiers[i], m['accuracy'], m['macro avg']['f1-score'], m['weighted avg']['f1-score']
    rows.append(r)

In [22]:
df_metrics = pd.DataFrame(rows, columns=('classifiers', 'accuracy', 'macro f1', 'weighted f1'))
df_metrics

,classifiers,accuracy,macro f1,weighted f1
0,NB,0.258616,0.005787,0.110225
1,DT,0.267168,0.023476,0.131643
2,RF,0.265937,0.026461,0.130471


In [23]:
df_metrics.to_clipboard()

## ANOVA

In [31]:
import pingouin as pg
from sklearn.metrics import accuracy_score, f1_score

In [27]:
cls_ = 'NB'
files = []
for i in range(10,):
    fl = glob.glob(f'./results/{cls_}/CV_{i}/*csv')[0]
    files.append(fl)

In [84]:
def get_fold_metrics(path):
    df = pd.read_csv(path)
    
    acc = accuracy_score(df.label, df.prediction)
    f1w = f1_score(df.label, df.prediction, average='weighted')
    f1m = f1_score(df.label, df.prediction, average='macro')
    
    return acc, f1w, f1m

In [85]:
def cls_cv_metrics(cls_):
    files = []
    for i in range(10,):
        fl = glob.glob(f'./results/{cls_}/CV_{i}/*csv')[0]
        files.append(fl)
        
    rows = []
    for i, fl in enumerate(files):
        r = get_fold_metrics(fl)
        rows.append((i, *r))
    
    return rows

In [86]:
rows = []
for cls_ in classifiers:
    r = cls_cv_metrics(cls_)
    rows.extend([ (cls_, *x) for x in r])

In [89]:
df = pd.DataFrame(rows, columns=('classifier', 'fold', 'accuracy', 'f1w', 'f1m'))
df.head()

,classifier,fold,accuracy,f1w,f1m
0,NB,0,0.260059,0.113768,0.006212
1,NB,1,0.263390,0.118484,0.006190
2,NB,2,0.259658,0.113812,0.006265
3,NB,3,0.261097,0.118047,0.006543
4,NB,4,0.266184,0.119394,0.006398


In [71]:
pg.anova(data=df, dv='accuracy', between='classifier')

,Source,ddof1,ddof2,F,p-unc,np2
0,classifier,2,27,2.191273,0.131256,0.139649


In [73]:
pg.anova(data=df, dv='f1w', between='classifier')

,Source,ddof1,ddof2,F,p-unc,np2
0,classifier,2,27,19.204564,0.000006,0.587214


In [90]:
pg.anova(data=df, dv='f1m', between='classifier')

,Source,ddof1,ddof2,F,p-unc,np2
0,classifier,2,27,26.341367,4.518495e-07,0.661156


In [93]:
pg.pairwise_tukey(data=df, dv='f1w', between='classifier')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,DT,NB,0.12819,0.116630,0.011560,0.002416,4.784486,0.001000,2.049278
1,DT,RF,0.12819,0.130653,-0.002463,0.002416,-1.019262,0.560676,-0.436568
2,NB,RF,0.11663,0.130653,-0.014023,0.002416,-5.803748,0.001000,-2.485845


In [94]:
pg.pairwise_tukey(data=df, dv='f1m', between='classifier')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,DT,NB,0.013794,0.006620,0.007174,0.001384,5.184603,0.001000,2.220655
1,DT,RF,0.013794,0.016294,-0.002500,0.001384,-1.806785,0.171033,-0.773877
2,NB,RF,0.006620,0.016294,-0.009674,0.001384,-6.991389,0.001000,-2.994532


In [96]:
pg.pairwise_tukey(data=df, dv='accuracy', between='classifier')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,DT,NB,0.265082,0.260391,0.004691,0.002933,1.599135,0.249014,0.684937
1,DT,RF,0.265082,0.266168,-0.001087,0.002933,-0.370470,0.900000,-0.158678
2,NB,RF,0.260391,0.266168,-0.005777,0.002933,-1.969604,0.123647,-0.843615
